In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

Failed `CDLL(/Library/Frameworks/GEOS.framework/Versions/Current/GEOS)`
Failed `CDLL(/opt/local/lib/libgeos_c.dylib)`


In [93]:
#1st - read in the csv file
data = pd.read_csv("small_scale_data.csv")
#pull out only the shape column which are all in string format
shape_col = data["SHAPE"]
usage_col = data["LU05_DESC"]
print (shape_col.shape)
print (usage_col.shape)

(23,)
(23,)


In [3]:
#create multigon finder and deal with it separately, then .apply(get_coord)
def find_multi(row):
    found = False
    if '), (' in row:
        found = True
    return found

In [4]:
#PREP FUNCTION COMES BEFORE FEEDING INTO THE APPROPRIATE MULTI/POLY FUNCTION TO MAKE DATA UNIFORM FORMAT
def prep_row(row):
    cleaned = row[16:-3]
    return cleaned # should look like 'lon lat, lon lat,...), (lon lat, lon lat...), (.....'

In [5]:
#GET_POLY FUNCTION CLEANS AND DEALS WITH SINGLE POLYGONS
#MULTIPOLYGON ROWS GET FED INTO GET_MULTI, WHICH THEN UTILIZES THIS FUNCTION WITHIN IT
def get_poly(data):
    # split the string into elements of a list by each comma
    vertex = data.split(",")
    # strip white spaces from each element
    vertex = [item.strip() for item in vertex ]
    # format the elements from 'latitude longitude' string to (latitude, longitude) floats
    vertex = [(float(item.split(" ")[0]) ,float(item.split(" ")[1])) for item in vertex]
    return (np.asarray(vertex))

In [6]:
#GET_MULTI DEALS WITH MULTIPOLYGONS BY UTILIZING GET_POLY
def get_multi(multi):
    #replace '), (' which separate polygons to ')&(' to be able to split properly
    fxn = multi.replace("), (", "&")
    #should look like : '(lon lat, ...)&(lon lat,...)&(lon lat,...)'
    #split polygons at the ampersand,  leaving strings ready to feed into get_poly
    fxn = fxn.split("&") 
    #should look like : ['(lon lat,...)', '(lon lat,...)', .....] 
    #create an empty 2D array to store a list of points per polygon, while still keeping together
    multi_2d = []
    #feed each polygon(i.e. each element in current fxn list) into the get_poly fxn so that each 
    #polygon has it's own formatted list of points while keeping them all together
    for i in range(len(fxn)):
        multi_2d.append(get_poly(fxn[i])) 
        multi_list = np.asarray(multi_2d)
    return multi_list


In [7]:
#SCRATCH CODE THAT HAS BEEN REMOVED        

#     vertex = data.split(" ")[1:]
#     print vertex[1::2]
#     first = vertex[0].split("(")[3]
#     last = vertex[-1].split(")")[0]
#     body = vertex[1:-1].split(",")
#     vertex = [first] + vertex[1:-1] + [last]
#     print vertex
#     print map(float, vertex)
#     y = map(float, vertex)
#     y = zip(*[iter(y)]*2)
    
#     print y

#print get_coord(vertex)
# final = vertex.apply(get_coord)
# print get_coord(vertex[16])

In [8]:
#SCALE FUNCTION
def scale(num):
    #create 14 digits by padding 0's if needed
    num1 = str(abs(num[0]))
    num2 = str(abs(num[1]))
    
    if len(num1) < 15:
        num1 = num1.ljust(15, '0')
    elif len(num1) > 15:
        num1 = num1[:15]
    
    if len(num2) < 15:
        num2 = num2.ljust(15, '0')
    elif len(num2) > 15:
        num2 = num2[:15]
    
    #shift decimal for 8 decimal places
    return np.array([-1.*(float(num1) * (10**5)),-1.*(float(num2) * (10**5))])

In [9]:
#CENTER FUNCTION
def center(coord):
    # coord is an np array containing lat and long
    
    #find mean
    mean = coord.mean()
    
    #center by subtracting the mean
    coord = coord - mean
    return coord

In [10]:
#GENERATE THE GRID LINES


#FINDING RANDOM POINTS
def find_randpoints(ranges1, ranges2):
    #ranges is an array of either lat or long
    point1 = np.random.uniform(ranges1[0], ranges1[1], 4)
    point2 = np.random.uniform(ranges2[0], ranges2[1], 4)
    return point1, point2

#6th - Determine where the random points lie in which boundaries (using package)

#7th - Set a percentage and determine point location

In [11]:
#Testing
# (long upper, long lower)
coord1 = np.array([-71.13748, -71.14003])
#(lat upper, lat lower)
coord2 = np.array([42.35827, 42.3601])

lon_scaled = scale(coord1)
lon_centered = center(lon_scaled)

lat_scaled = scale(coord2)
lat_centered = center(lat_scaled)

# print find_randpoints(lon_centered, lat_centered)
# print lon_centered[0],lon_centered[1]
# print lat_centered[0], lat_centered[0]

# print find_randpoints(coord1, coord2)
rand_pt1, rand_pt2 = find_randpoints(coord1, coord2)
# print scale(rand_pt1)
scaled1 = scale(rand_pt1)
# print center(scaled1)

# print rand_pt2
# print scale(rand_pt2)
scaled2 = scale(rand_pt2)
# print center(scaled2)

In [12]:
def parse_shapes(one_row):
    prepped = prep_row(one_row)
    is_multi = find_multi(prepped)
    if is_multi == True:
        prepd_split = get_multi(prepped)
        return (prepd_split)
    else:
        prepd_cleand = get_poly(prepped)
        return (prepd_cleand)

In [13]:
poly_test = (parse_shapes(shape_col[10]))

In [65]:
#APPLY PARSE_SHAPES METHOD TO ALL ELEMENTS OF SHAPE_COL LIST
shape_data_pretty = shape_col.apply(parse_shapes)

In [66]:
lat_rand = np.array([-71.13794487, -71.13926144, -71.13893327, -71.13919666])
lon_rand = np.array([42.35945246, 42.35927083, 42.35915658, 42.35879337])

In [67]:
ran_point0 = Point(lat_rand[0], lon_rand[0])
ran_point1 = Point(lat_rand[1], lon_rand[1])
ran_point2 = Point(lat_rand[2], lon_rand[2])
ran_point3 = Point(lat_rand[3], lon_rand[3])
polygon = Polygon(shape_data_pretty[10])
print(polygon.contains(ran_point3))

False


In [68]:
#now we want to write a loop to test point inside each of the gon, then when it's true 
#need to go back to csv file and check the type of land use attribute 



In [69]:
def is_multi(row):
    yes_multi = True
    if len(row.shape) > 1:
        yes_multi = False
    return yes_multi

In [70]:
#FUNCTION TO TEST POLYGONS
def test_point (point, cleaned_data):
    test_pt = Point(point[0], point[1]) 
    positive_gons = []
    for i in range(len(cleaned_data)):
        test_multi = is_multi(cleaned_data[i])
        if test_multi:
            for j in range(len(cleaned_data[i])):
                polygon1 = Polygon(cleaned_data[i][j])
                in_poly = polygon1.contains(test_pt)
                if in_poly == True:
                    positive_gons.append(i)
        else:
            in_poly = False
            polygon1 = Polygon(cleaned_data[i])
            in_poly = polygon1.contains(test_pt)
            if in_poly == True:
                positive_gons.append(i)
    return positive_gons

In [71]:
print (test_point([-71.13919666, 42.35879337], shape_data_pretty))

[14]


In [90]:
#FINAL FUNCTION FOR TESTING POINTS
def which_polygons(zipped_pts, nice_data):
    where_pts_r = []
    for i in range(len(zipped_pts)):
        where_pts_r.append(test_point(zipped_pts[i], nice_data))
    return where_pts_r

In [91]:
some_points = [[-71.13794487, 42.35945246],
               [-71.13926144, 42.35927083],
               [-71.13893327, 42.35915658],
               [-71.13919666, 42.35879337]]
print (some_points[0])

[-71.13794487, 42.35945246]


In [125]:
def find_usage(found_polygons):
    usage = []
    for i in range(len(found_polygons)):
        usage.append(usage_col[found_polygons[i][0]])
    return usage

In [126]:
got_em = which_polygons(some_points, shape_data_pretty)
print (find_usage(got_em))


['Industrial', 'Industrial', 'Industrial', 'Commercial']
